https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [1]:
import scipy.stats as sci_stats
from pathlib import Path
import numpy as np
import json
from os.path import join

In [2]:
BEATS = ['initial', 'final']

def convert_to_float(data):
    return np.array(data, dtype=float)

In [58]:
def present_values(folder, map_type, params=None):
    data = {}
    for i, beat in enumerate(BEATS):
        if beat not in data.keys():
            data[beat] = []
        path_to_results = folder / f"{beat}_{map_type}_map_metrics.json"
        with open(path_to_results, 'r') as file:
            sal_initial = json.load(file)
        values = convert_to_float(sal_initial['values'])
        if params is not None:
            where = []
            for i in range(len(sal_initial[params[0]])):
                if sal_initial[params[0]][i] == params[1]:
                    where.append(i)
            values = values[where]
        print(f"Mean value of {beat} beat: {np.nanmean(values)*100:.2f} +- {np.nanstd(values)*100:.2f}%")
        data[beat].append(values[np.logical_not(np.isnan(values))]*100)

    return data

def get_values_params(folder, map_type, beat, params=None):
    data = {}
    for i, label in enumerate(params[1]):
        if label not in data.keys():
            data[label] = []
        path_to_results = folder / f"{beat}_{map_type}_map_metrics.json"
        with open(path_to_results, 'r') as file:
            sal_initial = json.load(file)
        values = convert_to_float(sal_initial['values'])
        if params is not None:
            where = []
            for i in range(len(sal_initial[params[0]])):
                if sal_initial[params[0]][i] == label:
                    where.append(i)
            values = values[where]
        print(f"Mean value of {beat} beat: {np.nanmean(values)*100:.2f} +- {np.nanstd(values)*100:.2f}%")
        data[label].append(values[np.logical_not(np.isnan(values))]*100)

    return data

In [4]:
XAI_metrics_folder = Path("../XAI_metrics/metrics")

In [87]:
distributions = present_values(XAI_metrics_folder, "grad_cam", params=['pred_results', 'wrong'])
ttest = sci_stats.ttest_ind(np.ravel(distributions['final']), np.ravel(distributions['initial']), nan_policy='omit')
ttest

Mean value of initial beat: 8.33 +- 10.79%
Mean value of final beat: 10.21 +- 15.18%


Ttest_indResult(statistic=6.073532030383544, pvalue=1.3166591064715571e-09)

In [73]:
distributions = get_values_params(XAI_metrics_folder, "grad_cam", beat='initial', params=['true_labels', ['normal', 'abnormal']])
ttest = sci_stats.ttest_ind(np.ravel(distributions['normal']), np.ravel(distributions['abnormal']), nan_policy='omit')
ttest

Mean value of initial beat: 8.50 +- 4.20%
Mean value of initial beat: 10.65 +- 10.15%


Ttest_indResult(statistic=-28.051440032990268, pvalue=8.760368999796072e-172)

In [6]:
with open(XAI_metrics_folder / "final_saliency_map_metrics.json") as f:
    data = json.load(f)

In [7]:
grad_cam_final = np.array(data["values"], dtype=float)
grad_cam_final.shape

(49595,)

In [8]:
with open(XAI_metrics_folder / "initial_saliency_map_metrics.json") as f:
    data = json.load(f)

In [9]:
grad_cam_initial = np.array(data["values"], dtype=float)

In [10]:
ttest = sci_stats.ttest_ind(grad_cam_final, grad_cam_initial, nan_policy='omit')

In [11]:
ttest.pvalue

0.0

In [12]:
np.nanmean(grad_cam_initial), np.nanmean(grad_cam_final), np.nanstd(grad_cam_initial), np.nanstd( grad_cam_final), len(grad_cam_initial), len(grad_cam_final)

(0.2501896590565084,
 0.30340009211263913,
 0.18227530207732132,
 0.09172775423852264,
 49596,
 49595)